In [1]:
# 先测试PIL是否正常
from PIL import Image
print("PIL导入成功")

# 然后测试CLIP
import clip
print("CLIP导入成功")

PIL导入成功
CLIP导入成功


In [2]:
# 测试基础包的导入
try:
    from PIL import Image
    print("PIL导入成功")
except ImportError as e:
    print("PIL导入失败:", e)

try:
    import clip
    print("CLIP导入成功")
except ImportError as e:
    print("CLIP导入失败:", e)

# 打印版本信息以便调试
import PIL
print("PIL版本:", PIL.__version__)

PIL导入成功
CLIP导入成功
PIL版本: 7.1.2


In [3]:
import torch
import clip
from PIL import Image

# 打印 CLIP 版本和位置以确认我们使用的是正确的包
import inspect
print("CLIP location:", inspect.getfile(clip))

# 检查设备
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 加载模型
print("Loading CLIP model...")
model, preprocess = clip.load("ViT-B/32", device=device)
print("Model loaded successfully!")

CLIP location: /root/ENTER/envs/clip/lib/python3.7/site-packages/clip/__init__.py
Using device: cuda
Loading CLIP model...
Model loaded successfully!


In [4]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

image = preprocess(Image.open("wideCat.png")).unsqueeze(0).to(device)
text = clip.tokenize(["a diagram", "a dog", "a cat"]).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]



Label probs: [[0.00902 0.0856  0.9053 ]]


In [5]:
import os
import clip
import torch
from PIL import Image

# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

# Load and preprocess the image
image = Image.open("BlueCandyCatGirl.png") # 使用本地图片
image_input = preprocess(image).unsqueeze(0).to(device)

# Prepare the text inputs
classes = ["cat", "dog", "bird", "fish", "horse", "elephant", "monkey", "tiger", "lion", "bear"]
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in classes]).to(device)

# Calculate features
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

# Pick the top 5 most similar labels for the image
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
values, indices = similarity[0].topk(5)

# Print the result
print("\nTop predictions:\n")
for value, index in zip(values, indices):
    print(f"{classes[index]:>16s}: {100 * value.item():.2f}%")


Top predictions:

            fish: 16.54%
             cat: 15.78%
            lion: 14.83%
             dog: 10.35%
          monkey: 9.58%


In [6]:
import os
import clip
import torch
from torchvision.datasets import CIFAR100
from PIL import Image

# Load CIFAR100 dataset for its classes
cifar100 = CIFAR100(root='./data', download=True, train=False)

# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

# Load and preprocess the image
image = Image.open("wideCat.png") # 使用本地图片
image_input = preprocess(image).unsqueeze(0).to(device)

# Prepare the text inputs
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in cifar100.classes]).to(device)

# Calculate features
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

# Pick the top 5 most similar labels for the image
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
values, indices = similarity[0].topk(5)

# Print the result
print("\nTop predictions:\n")
for value, index in zip(values, indices):
    print(f"{cifar100.classes[index]:>16s}: {100 * value.item():.2f}%")

Files already downloaded and verified

Top predictions:

          lizard: 8.34%
         hamster: 7.95%
          possum: 7.02%
          turtle: 4.98%
           mouse: 4.46%


In [7]:
import os
import clip
import torch
from torchvision.datasets import CIFAR100
from PIL import Image

img_path = 'CLIP.png'
classes = ['cup', 'not_cup']




#准备输入集
image = Image.open(img_path)
image_input = preprocess(image).unsqueeze(0).to(device)
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in classes]).to(device) #生成文字描述

#特征编码
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

#选取参数最高的标签
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1) #对图像描述和图像特征  
values, indices = similarity[0].topk(1)

#输出结果
print("\nTop predictions:\n")
print('classes:{} score:{:.2f}'.format(classes[indices.item()], values.item()))




Top predictions:

classes:not_cup score:0.93
